## LangSmith Trace Span Evaluation

In [ ]:
%pip install grounded-ai

In [1]:
import os

from pydantic import BaseModel

from grounded_ai import Evaluator
from grounded_ai.otel import TraceConverter

# 1. Define the Raw Trace Data (simulating a LangSmith export)
# Note: Input contains null content for tool calls, testing our recent fix.
raw_trace = {
    "id": "run-root",
    "name": "AgentExecutor",
    "run_type": "chain",
    "inputs": {
        "messages": [{"type": "user", "content": "Find the 2007 US PGA winner"}]
    },
    "outputs": {"output": "Tiger Woods won the 2007 US PGA Championship"},
    "start_time": "2024-01-27T10:30:00.000Z",
    "end_time": "2024-01-27T10:30:10.000Z",
    "child_runs": [
        # 1. First LLM Call: INCORRECTLY decides to check weather instead of golf
        {
            "id": "run-llm-1",
            "name": "gpt-4",
            "run_type": "llm",
            "inputs": {
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": "Find the 2007 US PGA winner"},
                ]
            },
            "outputs": {
                "generations": [
                    [
                        {
                            "text": "",
                            "message": {
                                "role": "assistant",
                                "content": None,
                                "tool_calls": [
                                    {
                                        "id": "call-bad-1",
                                        "function": {
                                            "name": "search",
                                            "arguments": '{"query": "weather in Tokyo"}',
                                        },
                                    }
                                ],
                            },
                        }
                    ]
                ]
            },
            "start_time": "2024-01-27T10:30:01.000Z",
            "end_time": "2024-01-27T10:30:03.000Z",
            "extra": {"token_usage": {"prompt_tokens": 50, "completion_tokens": 20}},
        },
        # 2. Tool Execution: Returns irrelevant info
        {
            "id": "run-tool-1",
            "name": "search",
            "run_type": "tool",
            "inputs": {
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant"},
                    {"role": "user", "content": "Find the 2007 US PGA winner"},
                    {
                        "role": "assistant",
                        "tool_calls": [
                            {
                                "id": "call_abc123",
                                "type": "function",
                                "function": {
                                    "name": "search",
                                    "arguments": '{"query": "2007 US PGA winner"}',
                                },
                            },
                            {
                                "id": "call_def456",
                                "type": "function",
                                "function": {
                                    "name": "calculator",
                                    "arguments": '{"expression": "2007 + 17"}',
                                },
                            },
                        ],
                    },
                    {
                        "role": "tool",
                        "content": "Tiger Woods won the 2007 PGA Championship at Southern Hills",
                        "tool_call_id": "call_abc123",
                    },
                ]
            },
            "outputs": {
                "choices": [
                    {
                        "message": {
                            "role": "assistant",
                            "content": "Tiger Woods won the 2007 US PGA Championship",
                        }
                    }
                ]
            },
            "start_time": "2024-01-27T10:30:05.100000Z",
            "end_time": "2024-01-27T10:30:07.000000Z",
            "parent_run_id": "parent-run-uuid",
            "trace_id": "parent-run-uuid",
            "dotted_order": "20240127T103000000000Z.parent.llm2",
        },
    ],
}

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [3]:
# 1. Convert Raw Dict -> AgentTrace Object
trace = TraceConverter.from_langsmith(raw_trace).get_full_conversation()


# 2. Define Output Schema
class TraceEval(BaseModel):
    tool_usage_correct: bool
    issues: list[str]
    reasoning: str

# 4. Initialize Evaluator
evaluator = Evaluator(
    model="openai/gpt-4o",
    system_prompt="Analyze this agent execution trace for correctness.",
)

result = evaluator.evaluate(response=str(trace), output_schema=TraceEval)

In [4]:
from pprint import pprint

pprint(result.model_dump_json(indent=2))

('{\n'
 '  "tool_usage_correct": false,\n'
 '  "issues": [\n'
 '    "Misalignment in tool usage: The tool call is searching for \'weather in '
 'Tokyo\', whereas the user asked for the 2007 US PGA winner.",\n'
 '    "The assistant did not produce a correct response because it used an '
 'inaccurate search query not aligned with the user\'s question."\n'
 '  ],\n'
 '  "reasoning": "The agent was tasked with providing information about the '
 '2007 US PGA winner. Instead of searching for this specific information, the '
 'agent issued a tool call related to weather information in Tokyo. This '
 "indicates a mismatch between the user's request and the tool usage, "
 'resulting in incorrect or irrelevant information retrieval. For correct '
 "execution, the tool call should align with the user's query, i.e., it should "
 'search for information about the 2007 US PGA winner."\n'
 '}')
